In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from train_predict_utils import *

In [3]:
class LSTMModel_1(nn.Module):
    # basic one with two linear layers and final output with sigmoid
    def __init__(
        self, input_size, lstm_hidden_size=1024, linear_layer_size=4096, num_layers=1
    ):
        super().__init__()
        self.input_size = input_size
        self.num_layers = num_layers
        self.hidden_size = lstm_hidden_size
        self.lstm = nn.LSTM(
            input_size,
            lstm_hidden_size,
            batch_first=True,
            dropout=0,
            num_layers=num_layers,
        )
        self.fc1 = nn.Linear(lstm_hidden_size, linear_layer_size)
        self.fc2 = nn.Linear(linear_layer_size, linear_layer_size)
        self.fc3 = nn.Linear(linear_layer_size, input_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        # hidden state
        h_0 = torch.zeros(self.num_layers * 1, x.size(0), self.hidden_size).to(
            x.device
        )  # Modified line
        # cell state
        c_0 = torch.zeros(self.num_layers * 1, x.size(0), self.hidden_size).to(
            x.device
        )  # Modified line

        out, (hn, cn) = self.lstm(x, (h_0, c_0))
        out = self.fc1(out[:, -1, :])
        out = self.relu(out)
        out = self.fc3(out)
        out = torch.sigmoid(out)

        return out


In [9]:
data_dir = "../processed_dataset/"
train_dataset = CustomSequence(data_dir, range(2), file_batch_size=1, model_batch_size=512)
val_dataset = CustomSequence(data_dir, [1], file_batch_size=1, model_batch_size=512)


In [10]:
len(train_dataset)

2

In [11]:
# 1892 * 4 + 348 * 2
model = LSTMModel_1(input_size=8264)

In [12]:
train(model, train_dataset, num_epochs=10, val_dataset=val_dataset, use_gpu=True, data_parallel=True, out_dir=".", model_name="model", verbose=1, save_checkpoints=True, custom_loss=None)

Epoch 1 of 10
Epoch 1: Train Loss=0.145181387662887573, Val Loss=0.057540912181138992
Epoch 2 of 10
Epoch 2: Train Loss=0.056456286460161209, Val Loss=0.044879958033561707
Epoch 3 of 10
Epoch 3: Train Loss=0.039471747353672981, Val Loss=0.026325631886720657
Epoch 4 of 10
Epoch 4: Train Loss=0.021488886326551437, Val Loss=0.014857287518680096
Epoch 5 of 10
Epoch 5: Train Loss=0.011351361870765686, Val Loss=0.013296050019562244
Epoch 6 of 10
Epoch 6: Train Loss=0.010871140751987696, Val Loss=0.012453669682145119
Epoch 7 of 10
Epoch 7: Train Loss=0.010995287448167801, Val Loss=0.011442775838077068
Epoch 8 of 10
Epoch 8: Train Loss=0.010695511475205421, Val Loss=0.009264889173209667
Epoch 9 of 10
Epoch 9: Train Loss=0.009273342788219452, Val Loss=0.007836075499653816
Epoch 10 of 10
Epoch 10: Train Loss=0.008656975813210011, Val Loss=0.007580926176160574


(LSTMModel_1(
   (lstm): LSTM(8264, 1024, batch_first=True)
   (fc1): Linear(in_features=1024, out_features=4096, bias=True)
   (fc2): Linear(in_features=4096, out_features=4096, bias=True)
   (fc3): Linear(in_features=4096, out_features=8264, bias=True)
   (relu): ReLU()
 ),
 [0.14518138766288757,
  0.05645628646016121,
  0.03947174735367298,
  0.021488886326551437,
  0.011351361870765686,
  0.010871140751987696,
  0.010995287448167801,
  0.010695511475205421,
  0.009273342788219452,
  0.00865697581321001],
 [0.05754091218113899,
  0.04487995803356171,
  0.026325631886720657,
  0.014857287518680096,
  0.013296050019562244,
  0.012453669682145119,
  0.011442775838077068,
  0.009264889173209667,
  0.007836075499653816,
  0.007580926176160574])

In [15]:
test_dataset = CustomSequence(data_dir, [2], file_batch_size=1, model_batch_size=512, test_mode=True)

In [14]:
predict(model, model_param_path="model_epoch_2.pth", test_dataset=test_dataset, use_gpu=True, data_parallel=False, output_dir=".", output_name="all_preds.npy", verbose=1)

Predicting batch 1/2


FileNotFoundError: [Errno 2] No such file or directory: '../processed_dataset/X_new_90.npy'